# How to use

1) Choose a bandwidth:

L = 1

2) Define an array of planets that you'd like to use in the tests:

0: HD-189733 b, 1: WASP-33 b, 2: KELT-9 b, 3: WASP-12 b

To test all use:

planets = [0,1,2,3]

3) Choose the number of tests you'd like to perform:

num = 15

Now combine them an run the tests: 

run(L, planets, num)

In [ ]:
import numpy as np
import slepian
import matplotlib.pyplot as plt
from starry.kepler import Primary, Secondary, System
from scipy.optimize import minimize
import copy
import emcee
import corner
import csv
np.random.seed()

inferno = ["#E8FA5BFF","#EFE350FF","#F7CB44FF","#F9B641FF","#F9A242FF","#F68F46FF","#EB8055FF","#DE7065FF","#CC6A70FF","#B8627DFF","#A65C85FF","#90548BFF","#7E4E90FF","#6B4596FF","#593D9CFF","#403891FF","#253582FF","#13306DFF","#0C2A50FF","#042333FF"]  
infernodd = ["#E8FA5BFF","#F7CB44FF","#F9A242FF","#EB8055FF","#CC6A70FF","#A65C85FF","#7E4E90FF","#593D9CFF","#253582FF","#0C2A50FF"]  
      
plt.rcParams['figure.dpi'] = 125
plt.rcParams['axes.prop_cycle'] = plt.cycler(color = infernodd)

In [ ]:
def changeOfBasis(L):
    """Change of basis matrix."""
    
    return np.array(slepian.slepianToHarmonic(L), dtype = float)

In [ ]:
def preset(n):
    """Choose a planet."""
    
    # HD-189733 b:
    if n == 0:
        
        # Orbital Parameters:
        d = 0.001481 # Eclipse depth at 3.6 μm.
        r = 0.1504 # Planet radius in units of the stellar radii.
        a = 8.73 # Orbital semi-major axis in units of the stellar radii.
        inc = 85.690 # Orbital inclination in degrees (90 is an edge-on orbit).
        porb = 2.21857567 # Orbital period in days.
        prot = 2.21857567 # Rotation period in days (synchronous if prot = porb).
        ecc = 0 # Eccentricity.
        w = 90  # Longitude of pericenter in degrees (meaningless for ecc = 0).
        tref = 2458334.990899 # Time of transit in Julian days.
        noise = d/10
        
        t14 = 1.84/24 # Transit duration in days.
        
        # Errors:
        d_err = 0.000034
        r_err = 0.00385
        a_err = 0.15
        inc_err = 0.096
        porb_err = 0.00000015
        ecc_err = 0.0039
        w_err = 0
        tref_err = 0.0007535
        noise_err = noise/10
        
        # Time:
        time = np.linspace(tref + porb/2 - t14*1.5, tref + porb/2 + t14*1.5, 1000)
        
        # Assemble arrays.
        par_ges = np.array([d, r, a, inc, porb, ecc, w, tref, noise])
        
        par_err = np.array([d_err, r_err, a_err, inc_err, porb_err, ecc_err, w_err, tref_err, noise_err])
        
        return time, tref, porb, par_ges, par_err
        
    # WASP-33 b:
    elif n == 1:
        
        # Orbital Parameters:
        d = 0.003506 # Eclipse depth at 3.6 μm.
        r = 0.1118 # Planet radius in units of the stellar radii.
        a = 3.571 # Orbital semi-major axis in units of the stellar radii.
        inc = 86.63 # Orbital inclination in degrees (90 is an edge-on orbit).
        porb = 1.219870 # Orbital period in  days.
        prot = 1.219870 # Rotation period in days (synchronous if prot = porb).
        ecc = 0 # Eccentricity.
        w = 90  # Longitude of pericenter in degrees (meaningless for ecc = 0).
        tref = 2452984.82964 # Time of transit in Julian days.
        noise = d/10
            
        t14 = 2.854/24 # Transit duration days.
        
        # Errors:
        d_err = 0.000173
        r_err = 0.0002
        a_err = 0.010
        inc_err = 0.03
        porb_err = 0.000001
        ecc_err = 0
        w_err = 0
        tref_err = 0.00030
        noise_err = noise/10
        
        # Time:
        time = np.linspace(tref + porb/2 - t14*1.5, tref + porb/2 + t14*1.5, 1000)
        
        # Assemble arrays.
        par_ges = np.array([d, r, a, inc, porb, ecc, w, tref, noise])
        
        par_err = np.array([d_err, r_err, a_err, inc_err, porb_err, ecc_err, w_err, tref_err, noise_err])
        
        return time, tref, porb, par_ges, par_err
        
    # KELT-9 b
    elif n == 2:
        
        # Orbital Parameters
        #d = # Eclipse depth at 3.6 μm.
        r = 0.08228 # Planet radius in units of the stellar radii.
        a = 3.153 # Orbital semi-major axis in units of the stellar radii.
        inc = 86.79 # Orbital inclination in degrees (90 is an edge-on orbit).
        porb = 1.4811235 # Orbital period in days.
        prot = 1.4811235 # Rotation period in days (synchronous if prot =p orb).
        ecc = 0 # Eccentricity.
        w = 90  # Longitude of pericenter in degrees (meaningless for ecc = 0).
        tref = 2457095.68572 # Time of transit in Julian days.
        
        t14 = 3.9158/24 # Transit duration in days.
        
        # Errors:
        #d_err = 
        r_err = 0.00043
        a_err = 0.011
        inc_err = 0.25
        porb_err = 0.0000011
        ecc_err = 0
        w_err = 0
        tref_err = 0.00014
        
        # Time:
        time = np.linspace(tref + porb/2 - t14*1.5, tref + porb/2 + t14*1.5, 1000)
        
        # Assemble arrays.
        par_ges = np.array([d, r, a, inc, porb, ecc, w, tref, noise])
        
        par_err = np.array([d_err, r_err, a_err, inc_err, porb_err, ecc_err, w_err, tref_err, noise_err])
        
        return time, tref, porb, par_ges, par_err
        
    # WASP-12 b
    elif n == 3:
        
        # Orbital Parameters
        #d = # Eclipse depth at 3.6 μm.
        r = 0.1170 # Planet radius in units of the stellar radii.
        a = 3 # Orbital semi-major axis in units of the stellar radii.
        inc = 83.52 # Orbital inclination in degrees (90 is an edge-on orbit).
        porb = 1.09142 # Orbital period in days.
        prot = 1.09142 # Rotation period in days (synchronous if prot = porb).
        ecc = 0 # Eccentricity.
        w = 90  # Longitude of pericenter in degrees (meaningless for ecc = 0).
        tref = 2454508.97824 # Time of transit in Julian days.
        
        t14 = 3.0408/24 # Transit duration in days.
        
        # Errors:
        #d_err =
        r_err = 0.0002
        a_err = 0.00895
        inc_err = 0.03
        porb_err = 0.00000014432
        porb_err = 0
        ecc_err = 0.002
        w_err = 0
        tref_err = 0.00027
        
        # Time:
        time = np.linspace(tref + porb/2 - t14*1.5, tref + porb/2 + t14*1.5, 1000)
        
        # Assemble arrays.
        par_ges = np.array([d, r, a, inc, porb, ecc, w, tref, noise])
        
        par_err = np.array([d_err, r_err, a_err, inc_err, porb_err, ecc_err, w_err, tref_err, noise_err])
        
        return time, tref, porb, par_ges, par_err

In [ ]:
def randPar(par_ges, par_err):
    """Creates random true parameters based on the published (guess) values and their standard deviations."""
    
    # We'd like to use the published orbital parameters as guesses for the true values.
    # We set the true value to be random numbers within a distribution with standard
    # deviation given by the error on the published values.
    
    # Create the true values.
    par = np.random.randn(len(par_ges))*par_err + par_ges
    
    # Make sure that eccentricity is positive.
    while par[5] < 0:
        par[5] = np.random.randn()*par_err[5] + par_ges[5] 
    
    return par

In [ ]:
def initRand(L):
    """Creates a random map with luminosity set to one."""
    
    # Create random map array. We will set x00 using d.
    x = np.append([1], np.random.randn((L + 1)**2 - 1))
    
    # Make planet to test physicality.
    Test = Secondary(L)
    
    # Set coefficients.
    for l in range(1,L+1):
        Test[l,:] = x[(l)**2:(l + 1)**2]
    
    return Test.is_physical(), Test.flux(), x

def randMap(d, star_flux, L):
    """Creates a physical random map with luminosity set to one."""
    
    # Get a random map.
    is_physical, day_flux, x = initRand(L)
    
    # Keep trying until the map is phyiscal.
    while not is_physical:
        is_physical, day_flux, x = initRand(L)
    
    # Set x00 = L = Lp/Ls = d*star.flux()/planet.flux().
    return x*d*star_flux/day_flux

In [ ]:
def guessMap(x_tru, m_err, d_ges, star_flux, L):
    """Choose a guess based on the true map and the standard deviation."""
    
    # Re-normalize the map.
    x_tru = x_tru/x_tru[0]
    
    # Make guesses (we will set x00 using d_ges).
    x_ges = np.append([0],np.random.randn((L + 1)**2 - 1))*m_err + x_tru
    
    # Make planet to test flux.
    Test = Secondary(L)
    
    for l in range(1,L+1):
        Test[l, :] = x_ges[(l)**2 : (l + 1)**2]
    
    # Set x00_ges = L = Lp/Ls = d_ges*star.flux()/planet.flux().
    return x_ges*d_ges*star_flux/Test.flux()

In [ ]:
def setCoeffs(x, planet, L):
    """Takes a spherical harmonic vector X and sets the starry map parameters."""
    
    # Pull out luminosity.
    planet.L = x[0]
    
    # Normalize the spherical harmonics.
    x = x/planet.L 
    
    # Set the other map coefficients.
    for l in range(1, L + 1):
        planet[l, :] = x[(l)**2 : (l + 1)**2]
  
    return

In [ ]:
def setParams(par, planet):
    """Takes a parameter array and assigns them to a planet"""
    
    # Unpack parameters.
    d, r, a, inc, porb, ecc, w, tref, noise = par
    
    # Set parameters.
    planet.r = r
    planet.a = a
    planet.inc = inc
    planet.porb = porb
    planet.prot = porb
    planet.ecc = ecc
    planet.w = w
    planet.tref = tref
    
    return

In [ ]:
def fixGuess(par_ges, m_ges, par_err):
    """Removes all fixed terms from the guess."""
    
    # Fitting for d through x00*planet.flux()/star.flux(), so remove it.
    par_ges = np.delete(par_ges, 0) 
    par_err = np.delete(par_err, 0)
    
    # Remove fixed parameters
    par_ges = par_ges[par_err != 0]
    
    ges = np.concatenate((par_ges, m_ges), axis = None)
    
    return ges

In [ ]:
def parPrior(par):
    """Initial parameter priors."""
    
    # Check that the eccentricity is non-negative
    # Check that the flux error is positive.
    
    if  (np.any(par < 0)) or (par[7] == 0):
        return -np.inf
    else:
        return 0

In [ ]:
def gPrior(par, par_ges, par_err):
    """Gaussian prior probability."""
    
    # Find probability and ignore fixed terms,
    a = par - par_ges
    b = par_err
    c = np.divide(a, b, out = np.zeros_like(a), where = b != 0)
        
    return -0.5*np.sum(c**2)

In [ ]:
def lnLike(var, flux, par_ges, par_err, n, M, arg):
    """Log likelihood."""

    # Unpack the arguments:
    time, system, star, planet, L = arg
    
    # Split up the variables.
    par = var[:len(var) - n]
    m = var[len(var) - n:]
    
    # Change to spehrical harmonic basis
    x = np.dot(M, m)
    
    # Set priors:
    
    # Check that x00 > 0 so that luminosity is positive.
    if x[0] <= 0:
        return -np.inf
    
    # Add in any missing orbital parameters (except d).
    # Recall that if parameter i is fixed, par_ges[i] = par_tru[i].
    for i in range(1, len(par_err)):
        if par_err[i] == 0:
            par = np.insert(par, i-1, par_ges[i])
    
    # Check parameter priors.
    l = parPrior(par)
    
    if np.isinf(l):
        return l  
    
    # Set map coefficients
    setCoeffs(x, planet, L)
    
    # Add d to par:
    
    #If fixed:
    if par_err[0] == 0: 
        par = np.insert(par, 0, par_ges[0])
    
    #If not fixed:
    else:
        par = np.insert(par, 0, x[0]*planet.flux()/star.flux())
      
    # Check gaussian priors
    l += gPrior(par, par_ges, par_err)
                  
    # Set planet parameters.
    setParams(par, planet)
    
    # Compute flux.
    system.compute(time)
    flux_var = planet.lightcurve
    
    # Caluculate log likelihood.
    like = np.sum(((flux - flux_var)/par[8])**2 + 2*np.log(par[8]))
    l += -0.5*like
    
    return l

In [ ]:
def maxLike(flux, par_ges, m_ges, par_err, n, M, arg):
    """Maximum likelihood solution."""
    
    # Unpack the arguments:
    time, system, star, planet, L = arg
    
    # Assemble guess array and throw away parameters that are held fixed so we don't fit for them
    ges = fixGuess(par_ges, m_ges, par_err)
    
    # Define negative log likelihood
    nlnLike = lambda *args: -lnLike(*args)
    
    # Find the numerical optimum of the likelihood function
    soln = minimize(nlnLike, ges, args = (flux, par_ges, par_err, n, M, arg), method = 'Nelder-Mead')
    mls = soln.x
    
    # Split up the variables.
    par = mls[:len(mls) - n]
    m = mls[len(mls) - n:]
    
    x = np.dot(M, m)
    
    # Set map coefficients.
    setCoeffs(x, planet, L)
    
    # Add in any missing orbital parameters (except d).
    # Recall that if parameter i is fixed, par_guess[i] = par_true[i].
    for i in range(1, len(par_err)):
        if par_err[i] == 0:
            par = np.insert(par, i-1, par_ges[i])
    
    # Add d to par:
    
    #If fixed:
    if par_err[0] == 0: 
        par = np.insert(par, 0, par_gus[0])
    
    #If not fixed:
    else:
        par = np.insert(par, 0, x[0]*planet.flux()/star.flux())
      
    
    # Set planet parameters.
    setParams(par, planet)
    
    # Compute flux.
    system.compute(time)
    flux_mls = planet.lightcurve

    return mls, flux_mls

In [ ]:
def initPos(mls, par_ges, par_err, m_err, n, M, arg, walkers):
    """Initialization in a ball around the max likelihood solution."""
    
    # Unpack the arguments:
    time, system, star, planet, L = arg
    
    # Split up the variables.
    par_mls = mls[:len(mls) - n]
    m_mls = mls[len(mls) - n:]
    
    x_mls = np.dot(M, m_mls)
    
    # Get the max likelihood eclipse depth
    setCoeffs(x_mls, planet, L)

    d_mls = x_mls[0]*planet.flux()/star.flux()
    
    # Generate some random eclipse depths
    d_int = np.random.rand(walkers)*par_err[0] + d_mls 
    
    # Get initial map coefficients
    m_int = [np.dot(np.transpose(M), guessMap(x_mls, m_err, d_int[0], star.flux(), L))]
    
    for i in range(walkers - 1):
        m_int = np.append(m_int, [np.dot(np.transpose(M), guessMap(x_mls, m_err, d_int[i + 1], star.flux(), L))], axis = 0)
    
    # Get the error on the non-fixed parameters
    err = np.delete(par_err, 0)
    err = err[err != 0]
    
    # Get inital parameters.
    par_int = np.random.randn(walkers, len(par_mls))*err + par_mls
    
    # Make sure the the guesses are possible by throwing them through the prior
    for i in range(walkers):
        
        par_tmp = copy.deepcopy(par_int[i])
        
        # Add in any missing orbital parameters (except d).
        # Recall that if parameter j is fixed, par_guess[j] = par_true[j].
        for j in range(1, len(par_err)):
            if par_err[j] == 0:
                par_tmp = np.insert(par_tmp, j-1, par_ges[j])

        # Check parameter priors.
        l = parPrior(par_tmp)  
    
        while np.isinf(l):
            
            par_int[i] = np.random.randn()*err + par_mls
            
            par_tmp = copy.deepcopy(par_int[i])
        
            # Add in any missing orbital parameters (except d).
            for j in range(1, len(par_err)):
                if par_err[j] == 0:
                    par_tmp = np.insert(par_tmp, j-1, par_ges[j])

            # Check parameter priors.
            l = parPrior(par_tmp)  
    
    # Combine.
    init = np.append(par_int, m_int, axis = 1)
    
    return init

In [ ]:
def runMCMC(flux, mls, par_ges, par_err, m_err, n, M, arg, walkers, steps):
    
    # Unpack the arguments:
    time, system, star, planet, L = arg

    # Initialize around the max likelihood solution
    init = initPos(mls, par_ges, par_err, m_err, n, M, arg, walkers)
    
    # Run an MCMC chain
    sampler = emcee.EnsembleSampler(walkers, len(mls), lnLike, args = (flux, par_ges, par_err, n, M, arg))
    sampler.run_mcmc(init, steps, progress = True)
    samples = sampler.get_chain(discard = 1000, flat = True)
    chain = sampler.get_chain()
    
    # Get mc solution and mc error
    mcs = np.array([])
    mce = np.array([])
    for i in range(len(mls)):
        mcp = np.percentile(samples[:, i], [16, 50, 84])
        mcs = np.append(mcs,mcp[1])
        mce = np.append(mce,(np.abs(mcp[1] - mcp[0]) + np.abs(mcp[2] - mcp[1]))/2)
    
    if par_err[0] != 0: # If the eclipse depth is not fixed
        
        # Calculate the eclipse depth for mls
        m_mls = mls[len(mcs) - n:]
        x_mls = np.dot(M, m_mls)

        setCoeffs(x_mls, planet, L)

        d_mls = x_mls[0]*planet.flux()/star.flux()
        
        mls = np.insert(mls, 0, d_mls)
        
        # Calculate the eclipse depth for mcs
        
        index = np.random.randint(len(samples), size = 1000)
        d_mcs = np.array([])
        
        for i in index:

            m_mcs = samples[i][len(mcs) - n:]
            x_mcs = np.dot(M, m_mcs)

            setCoeffs(x_mcs, planet, L)
    
            d_mcs = np.append(d_mcs, x_mcs[0]*planet.flux()/star.flux())
        
        d_mcp = np.percentile(d_mcs, [16, 50, 84])
        mcs = np.insert(mcs, 0, d_mcp[1])
        mce = np.insert(mce, 0, (np.abs(d_mcp[1] - d_mcp[0]) + np.abs(d_mcp[2] - d_mcp[1]))/2)
        
    # Split up the variables (to get mcs flux)
    par = mcs[:len(mcs) - n]
    m = mcs[len(mcs) - n:]
    
    x = np.dot(M, m)
    
    if par_err[0] == 0:
        
        par = np.insert(par, 0, par_ges[0])

    # Add in any missing orbital parameters (except d).
    # Recall that if parameter j is fixed, par_ges[j] = par_tru[j].
    for i in range(1, len(par_err)):
        if par_err[i] == 0:
            par = np.insert(par, i, par_ges[i])
    
    # Set planet parameters.
    setCoeffs(x, planet, L)
    setParams(par, planet)
    
    # Compute flux.
    system.compute(time)
    flux_mcs = planet.lightcurve
    
    return mls, mcs, mce, flux_mcs, chain 

In [ ]:
def fit(flux, par_tru, m_tru, par_ges, m_ges, par_err, m_err, M, arg):
    
    # Unpack arguments:
    time, system, star, planet, L = arg
    
    # N is the number of map coefficients.
    n = (L + 1)**2 #1,4,9,16,25

    # Find the max likelihood solution and flux.
    mls, flux_mls = maxLike(flux, par_ges, m_ges, par_err, n, M, arg)

    # Run an MCMC chain.
    walkers = 2*len(mls)
    steps = 10000
    mls, mcs, mce, flux_mcs, chain = runMCMC(flux, mls, par_ges, par_err, m_err, n, M, arg, walkers, steps)
    
    # Get the true non-fixed paramters
    par_tru = np.array(par_tru)
    par_err = np.array(par_err)
    tru = np.concatenate((par_tru[par_err != 0], m_tru), axis = None)
    
    return tru, mls, flux_mls, mcs, flux_mcs, mce, chain

In [ ]:
def labels(par_err, L):
    
    # We don't need labels for d since we never plot it.
    par_err = np.delete(par_err, 0)
    
    # Parameter labels list.
    par = np.array(["r", "a", "inc", "porb", "ecc", "w", "tref", r"$\sigma$"])
    par = par[par_err != 0]
    par = par.tolist()
    
    labels_s = par.copy()
    labels_h = par.copy()
    
    # Add fucntion name.
    labels_s.insert(0, "Slepian")
    labels_h.insert(0, "Harmonic")
    
    # Map coefficient labels
    for i in range((2+1)**2):
        labels_s.append(r"$t_" + str(i+1) + "$")
        
    for l in range(L+1):
        for m in range(-l,l+1):
            labels_h.append(r"$x_{" + str(l) + "," + str(m) + "}$")
    
    return labels_s, labels_h

In [ ]:
def plots(flux, flux_tru, tru, par_ges, par_err, flux_mls, flux_mcs, M, arg, chain, labels, time, tref, porb, p, i):
    """Construct relevant plots."""
    
    # Unpack arguments:
    time, system, star, planet, L = arg
    
    # Ignore eclipse depth
    if par_ges[0] != 0:
        tru = np.delete(tru, 0)
    
    walkers = 2*len(tru)
    steps = 10000
    
    """MCMC chains plot."""
    
    fig, axes = plt.subplots(len(tru), figsize = (15, 10), sharex = True)
    for j in range(len(tru)):
        ax = axes[j]
        ax.plot(chain[:, :, j], alpha = 0.3, lw = 1)
        ax.set_xlim(0, len(chain))
        ax.set_ylabel(labels[j+1], fontsize = 16)
        ax.yaxis.set_label_coords(-0.1, 0.5)

    axes[-1].set_xlabel("Number of Steps", fontsize = 14)
    name = "plot_" + str(p) + "_" + str(i) + "_0_" + labels[0].lower()
    plt.savefig(name, dpi = 150)
    plt.clf()
    
    
    flat_chain = np.reshape(chain, (steps*walkers, len(tru)))
    flat_chain = flat_chain[2*len(tru)*1000:, :]
    
    
    """MCMC sample plot."""

    plt.figure(figsize = (10,10))
    plt.plot((time - (tref + porb/2))*60*24, flux, color = inferno[1], marker = '.', linestyle = 'None', label = 'Flux with noise')
    
    index = np.random.randint(len(flat_chain), size = 200)

    for n in index:

        par = flat_chain[n][:len(flat_chain[n]) - (L + 1)**2]
        m = flat_chain[n][len(flat_chain[n]) - (L + 1)**2:]
        x = np.dot(M, m)
        
        par = np.insert(par, 0, par_ges[0]) # This term does not change the lightcurve 

        # Add in any missing orbital parameters (except d).
        for j in range(1, len(par_err)):
            if par_err[j] == 0:
                par = np.insert(par, j, par_ges[j])

        setParams(par, planet)
        setCoeffs(x, planet, L)
    
        system.compute(time)
        flux = planet.lightcurve

        plt.plot((time - (tref + porb/2))*60*24, flux, color = inferno[4], marker = 'None', alpha = 0.1)
    
    plt.plot((time - (tref + porb/2))*60*24, flux_tru, color = inferno[9], marker = 'None', label = 'True Flux')
    plt.plot((time - (tref + porb/2))*60*24, flux_mls, color = inferno[14], marker = 'None', label = labels[0] + ' Maxlikelihood Flux')
    plt.plot((time - (tref + porb/2))*60*24, flux_mcs, color = inferno[19], marker = 'None', label = labels[0] + ' MCMC Flux')
    plt.legend(fontsize = 10)
    plt.xlabel("Time from Eclipse (minutes)", fontsize = 14)
    plt.ylabel("Planetary Flux", fontsize = 14)
    name = "plot_" + str(p) + "_" + str(i) + "_1_" + labels[0].lower()
    plt.savefig(name, dpi = 150)
    plt.clf()
    
    
    """Corner plot."""
    
    fig = corner.corner(flat_chain, labels = labels[1:], bins = 50, truths = tru);
    
    for ax in fig.axes:
        ax.xaxis.label.set_fontsize(20)
        ax.yaxis.label.set_fontsize(20)
    name = "plot_" + str(p) + "_" + str(i) + "_2_" + labels[0].lower()
    plt.savefig(name, dpi = 150)
    plt.clf()
    
    plt.close('all')

In [ ]:
def execute(L, p, i):
    """Retrieve presets, set truth, make guess then fit using Slepian and Harmonic functions."""

    # Identity Matrix.
    I = np.identity((L + 1)**2)

    # Slepian Matrix (Slepian to spherical harmonic).
    C = changeOfBasis(L)
    
    # Instantiate star-planet system.
    star = Primary()
    planet = Secondary(L)
    system = System(star, planet)

    # Retrieve presets: 0: HD-189733 b, 1: WASP-33 b, 2: KELT-9 b, 3: WASP-12 b
    time, tref, porb, par_ges, par_err = preset(p)
    
    # Define true parameters.
    par_tru = randPar(par_ges, par_err)

    # Choose a semi-random planetary map.
    x_tru = randMap(par_tru[0], star.flux(), L)
    
    # Get true map in Slepian basis.
    t_tru = np.dot(np.transpose(C), x_tru)
    
    # Set the standard deviation of the map coefficients in units of L.
    m_err = 1/10
    
    # Make a guess map.
    x_ges = guessMap(x_tru, m_err, par_ges[0], star.flux(), L)
    
    # Get guess map in Slepian basis.
    t_ges = np.dot(np.transpose(C), x_ges)
    
    # Generate the true light curve
    setCoeffs(x_tru, planet, L)
    setParams(par_tru, planet)
    
    system.compute(time)
    flux_tru = planet.lightcurve

    # Create noised flux
    flux = np.random.randn(len(time))*par_tru[8] + flux_tru
    
    # Generate the guess light curve
    setCoeffs(x_ges, planet, L)
    setParams(par_ges, planet)
    
    system.compute(time)
    flux_ges = planet.lightcurve
    
    # Define argument array
    arg = time, system, star, planet, L
        
    # Fit light curve
    tru_s, mls_s, flux_mls_s, mcs_s, flux_mcs_s, mce_s, chain_s = fit(flux, par_tru, t_tru, par_ges, t_ges, par_err, m_err, C, arg)
    tru_h, mls_h, flux_mls_h, mcs_h, flux_mcs_h, mce_h, chain_h = fit(flux, par_tru, x_tru, par_ges, x_ges, par_err, m_err, I, arg)
    
    # Set labels
    labels_s, labels_h = labels(par_err, L)
    
    # Plot
    plots(flux, flux_tru, tru_s, par_ges, par_err, flux_mls_s, flux_mcs_s, C, arg, chain_s, labels_s, time, tref, porb, p, i)
    plots(flux, flux_tru, tru_h, par_ges, par_err, flux_mls_h, flux_mcs_h, I, arg, chain_h, labels_h, time, tref, porb, p, i)
    
    # Results
    Flux = np.concatenate(([flux], [flux_tru], [flux_ges], [flux_mls_s], [flux_mls_h], [flux_mcs_s], [flux_mcs_h]), axis = 0)
    
    Sol_s = np.concatenate(([tru_s], [mls_s], [mcs_s], [mce_s]), axis = 0)
    Sol_h = np.concatenate(([tru_h], [mls_h], [mcs_h], [mce_h]), axis = 0)
    
    return Flux, Sol_s, Sol_h, time, tref, porb 

In [ ]:
def run(L, planets, num):    
    
    for p in planets:

        for i in range(num):

            Flux, Sol_s, Sol_h, time, tref, porb = execute(1, p, i)

            with open('solutions' + str(p) + '.csv','a',newline = '') as f:
                writer = csv.writer(f)
                writer.writerow(Sol_s[0])
                writer.writerow(Sol_s[1])
                writer.writerow(Sol_s[2])
                writer.writerow(Sol_s[3])
                writer.writerow(Sol_h[0])
                writer.writerow(Sol_h[1])
                writer.writerow(Sol_h[2])
                writer.writerow(Sol_h[3])

            plt.figure(figsize=(10,10))
            plt.plot((time - (tref + porb/2))*60*24, Flux[0], color = inferno[1], marker = '.', linestyle = 'None',label = 'Flux with noise')
            plt.plot((time - (tref + porb/2))*60*24, Flux[1], color = inferno[3], marker = 'None', label = 'True Flux')
            plt.plot((time - (tref + porb/2))*60*24, Flux[2], color = inferno[6], marker = 'None', label = 'Guess Flux')
            plt.plot((time - (tref + porb/2))*60*24, Flux[3], color = inferno[9], marker = 'None', label = 'Slepian Maxlikelihood Flux')
            plt.plot((time - (tref + porb/2))*60*24, Flux[4], color = inferno[12], marker = 'None', label = 'Harmonic Maxlikelihood Flux')
            plt.plot((time - (tref + porb/2))*60*24, Flux[5], color = inferno[15], marker = 'None', label = 'Slepian MCMC Flux')
            plt.plot((time - (tref + porb/2))*60*24, Flux[6], color = inferno[18], marker = 'None', label = 'Harmonic MCMC Flux')
            plt.legend(fontsize = 10)
            plt.xlabel("Time from Eclipse (minutes)", fontsize = 14)
            plt.ylabel("Planetary Flux", fontsize = 14)
            name = "plot_" + str(p) + "_" + str(i)
            plt.savefig(name, dpi = 150)
            plt.clf()
            plt.close('all')

    return

In [ ]:
L = 1
planets = [0,1]
num = 5

run(L, planets, num)